<a href="https://colab.research.google.com/github/samyam200200/python/blob/master/ANNwithGRIDSEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [64]:
#import
dataset=pd.read_csv("Churn_Modelling.csv")
X=dataset.iloc[:,3:13].values
Y=dataset.iloc[:,13].values

In [65]:
#one hot encoding variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Country column
ct = ColumnTransformer([("Geo", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)

# Male/Female
labelencoder_X = LabelEncoder()
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
X = X.astype('float64')

In [66]:
#removing 1 dummy varibale ie 0 
X=X[:,1:]

In [67]:
#Splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.20, random_state=42)

In [68]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [69]:
#Making ANN
# Importing Keras
import keras

In [70]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [71]:
#Initalizing ANN
classifier=Sequential()

In [72]:
# Adding first layer -  input layera and first hidden layer (secondly with dropout)
# units = output_dim (i.e. number of hidden layers equal to inputlayers(11)+outputlayers(1)/2)
# init=kernel_initializer (initial weights)
# input_dim (input layers)
# p - fraction of neuron to drop in each iteration (start with p = 0.1, then still overfitting increase p eventually. do not go over 0.5)
classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu", input_dim=11))
classifier.add(Dropout(p=0.1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys


In [73]:
#adding new layer
#this is added to previous neural network, its not needed to add input layer
classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu"))
classifier.add(Dropout(p=0.1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.


In [74]:
#output layer
#softmax for more than one output
classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

In [75]:
#Compiling ANN
#adam is for stochastic gradient descent
#loss function is for the weights, here for logistic regression we will use lograthmic loss
#more than 2 varibale categorical_crossentropy
#what is used to improve the goodness of model
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [76]:
# fitting ANN into the dataset
#also pass batch size
# and pass epoch (repeating previous spteps this many times)
c=classifier.fit(X_train, Y_train, batch_size=10, nb_epoch=100)
c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Epoch 1/100
8000/8000 [==============================] - 2s 305us/step - loss: 0.5008 - accuracy: 0.7939
Epoch 2/100
8000/8000 [==============================] - 2s 290us/step - loss: 0.4402 - accuracy: 0.7945
Epoch 3/100
8000/8000 [==============================] - 2s 290us/step - loss: 0.4379 - accuracy: 0.7945
Epoch 4/100
8000/8000 [==============================] - 2s 290us/step - loss: 0.4305 - accuracy: 0.7945
Epoch 5/100
8000/8000 [==============================] - 2s 284us/step - loss: 0.4298 - accuracy: 0.7945
Epoch 6/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.4294 - accuracy: 0.8094
Epoch 7/100
8000/8000 [==============================] - 2s 289us/step - loss: 0.4293 - accuracy: 0.8234
Epoch 8/100
8000/8000 [==============================] - 2s 286us/step - loss: 0.4313 - accuracy: 0.8259
Epoch 9/100
8000/8000 [==============================] - 2s 284us/step - loss: 0.4286 - accuracy: 0.8267
Epoch 10/100
8000/8000 [==============================]

In [77]:
# predicting in test set
Y_pred=classifier.predict(X_test)
Y_pred

array([[0.13837934],
       [0.10131505],
       [0.39086366],
       ...,
       [0.44727564],
       [0.04778833],
       [0.20792748]], dtype=float32)

In [78]:
#All greater than 0.5 is true less than 0.5 is false
Y_pred=(Y_pred>0.5)

In [79]:
#  creating confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test, Y_pred)
cm

array([[1560,   47],
       [ 277,  116]])

In [80]:
# new prediction
#in horizontal matrix (using double [[]]) - creating 2d array
new_prediction=classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction=(new_prediction>0.5)
new_prediction

array([[False]])

In [81]:
#K-fold cross validation
#Combine keras and scikitlearn - keras wrapper wraps kfold cross validation inside the keras package
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [82]:
#use a function for kerasclassifier - a fuction returning and building on ANN architecture
#def defines function - builds ANN classifier
#only use the part of code that builds the ANN not the last part
def build_classifier():
  classifier=Sequential()
  classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
  classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
  classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
  classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return classifier
#this returns a classifer for local and we need to make a global output using wrapper
classifier = KerasClassifier(build_fn =build_classifier, batch_size=10, nb_epoch=100)
#new obejct to contain the accuracy of 10 diffrent cross validation scores
#n_jobs for number of cpu used -1 is for all CPU
accuracies=cross_val_score(estimator = classifier, X = X_train, y=Y_train, cv=10, n_jobs = -1)

In [83]:
accuracies

array([0.8125    , 0.79250002, 0.78750002, 0.79124999, 0.79124999,
       0.82875001, 0.79624999, 0.76875001, 0.79374999, 0.80374998])

In [84]:
mean=accuracies.mean()
variance=accuracies.std()


In [85]:
#Improving ANN - dropout regularization
#Dropout - used to prevent overfitting
#High variance = overfitting
#Dropout causes random diabling of the neurons when training when they become too much correlated, 
#dependent to each other - dropout causes indendent correlation

In [86]:
#Parameter tuning - Number of epoch, batch size, optimizer, number of neurons in the layers
# Best value of these parameters using grid search
# Use keras classifier - instead of coss_val_score we use grid search
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
#new arguement created for optimizer and changed inside the function - do accordingly for others
def build_classifier(optimizer):
  classifier=Sequential()
  classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
  classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
  classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
  classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return classifier
#this returns a classifer for local and we need to make a global output using wrapper
classifier = KerasClassifier(build_fn =build_classifier)
#creating grid search 
#first create dictionary - keys and values - batch size, optimizer, epoch
parameters = {'batch_size':[25, 32],
              'nb_epoch':[100, 500],
              'optimizer':['adam', 'rmsprop']}
#creating grid search object from parameters
grid_search=GridSearchCV(estimator=classifier,
                         param_grid=parameters,
                         scoring='accuracy',
                         cv=10)
#fitting grid search to data
grid_search=grid_search.fit(X_train, Y_train)
#selacting best parameters
best_parameters=grid_search.best_params_
best_accuracy=grid_search.best_score_

Epoch 1/1
7200/7200 [==============================] - 1s 118us/step - loss: 0.5685 - accuracy: 0.7931
Epoch 1/1
7200/7200 [==============================] - 1s 118us/step - loss: 0.5586 - accuracy: 0.7940
Epoch 1/1
7200/7200 [==============================] - 1s 114us/step - loss: 0.6558 - accuracy: 0.7950
Epoch 1/1
7200/7200 [==============================] - 1s 113us/step - loss: 0.5541 - accuracy: 0.7950
Epoch 1/1
7200/7200 [==============================] - 1s 115us/step - loss: 0.5480 - accuracy: 0.7944
Epoch 1/1
7200/7200 [==============================] - 1s 117us/step - loss: 0.5642 - accuracy: 0.7907
Epoch 1/1
7200/7200 [==============================] - 1s 125us/step - loss: 0.5619 - accuracy: 0.7943
Epoch 1/1
7200/7200 [==============================] - 1s 120us/step - loss: 0.5629 - accuracy: 0.7969
Epoch 1/1
7200/7200 [==============================] - 1s 119us/step - loss: 0.5578 - accuracy: 0.7940
Epoch 1/1
7200/7200 [==============================] - 1s 121us/step - lo

In [87]:
best_accuracy

0.7945